In [0]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets.mnist import load_data

In [0]:
(x_train,y_train),(x_test,y_test) = load_data()

In [0]:
x_train = np.expand_dims(x_train,axis=-1)
x_test = np.expand_dims(x_test,axis=-1)

In [81]:
print(x_train.max())
x_train = x_train/255
x_test = x_test/255

255


In [0]:
x = tf.placeholder(tf.float32,shape=[None,28,28,1])
y = tf.placeholder(tf.int64,shape=[None])

In [83]:
y_onehot = tf.one_hot(y,10)
print(y_onehot)

Tensor("one_hot_4:0", shape=(?, 10), dtype=float32)


In [0]:
keep_prob = tf.placeholder(tf.float32)

In [0]:
W_conv1 = tf.Variable(tf.truncated_normal(shape=[5, 5, 1, 64], stddev=5e-2))

In [0]:
b_conv1 = tf.Variable(tf.constant(0.1, shape=[64]))

In [0]:
# W_conv1 5,5 필터와 x라는 이미지를 strides 1로 함. 텐서는 기본 4차원이기때문에 스트라이드가 4차원이며 기본적으로 중앙 값2개만 사용함.
# 활성화 함수로 relu를 사용하고, 1차원 64 bias를 더해줌.
h_conv1 = tf.nn.relu(tf.nn.conv2d(x, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1)

In [0]:
h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [0]:
# 5,5,64,64인 이유는 이전 W_conv1이 5,5,1,64이기 때문.
W_conv2 = tf.Variable(tf.truncated_normal(shape=[5, 5, 64, 64], stddev=5e-2))

In [0]:
b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))

In [0]:
h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2)

In [0]:
h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [0]:
# 여기가 7인 이유는, 처음에 x placeholder가 28x28이기때문에, 28에서 풀링 한번에 14, 한번 더하면 7이 되고 그 후 1차원 벡터로 만들어주는 작업이기 때문임.
W_fc1 = tf.Variable(tf.truncated_normal(shape=[7 * 7 * 64, 384], stddev=5e-2))
b_fc1 = tf.Variable(tf.constant(0.1, shape=[384]))
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob=keep_prob)

In [0]:
W_fc2 = tf.Variable(tf.truncated_normal(shape=[384, 10], stddev=5e-2))
b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))
logits = tf.matmul(h_fc1_drop,W_fc2) + b_fc2
y_pred = tf.nn.softmax(logits)


In [0]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_onehot, logits=logits))

In [0]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

In [0]:
correct_prediction = tf.equal(tf.argmax(y_pred, 1), y)

In [0]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [99]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for i in range(10000):
    idx = np.random.randint(x_train.shape[0], size=128)
    batch = (x_train[idx], y_train[idx])
    if i % 100 == 0:
      train_accuracy = sess.run(accuracy, feed_dict= {x: batch[0], y: batch[1], keep_prob: 1.})
      loss_print = loss.eval(feed_dict={x: batch[0], y: batch[1], keep_prob: 1.})
      print("step: %d, acc: %f, loss: %f" % (i, train_accuracy, loss_print))
    sess.run(train_step, feed_dict={x: batch[0], y: batch[1], keep_prob: 0.5})
  test_accuracy = accuracy.eval(feed_dict={x: x_test, y: y_test, keep_prob: 1.})
  print("test acc: %f" % test_accuracy)

step: 0, acc: 0.125000, loss: 2.324997
step: 100, acc: 0.898438, loss: 0.653130
step: 200, acc: 0.898438, loss: 0.359547
step: 300, acc: 0.890625, loss: 0.302674
step: 400, acc: 0.945312, loss: 0.187661
step: 500, acc: 0.945312, loss: 0.260525
step: 600, acc: 0.953125, loss: 0.141144
step: 700, acc: 0.976562, loss: 0.073207
step: 800, acc: 0.976562, loss: 0.162710
step: 900, acc: 0.937500, loss: 0.230690
step: 1000, acc: 0.968750, loss: 0.149236
step: 1100, acc: 0.992188, loss: 0.042140
step: 1200, acc: 0.960938, loss: 0.118689
step: 1300, acc: 0.984375, loss: 0.043819
step: 1400, acc: 0.968750, loss: 0.058135
step: 1500, acc: 0.984375, loss: 0.051105
step: 1600, acc: 0.960938, loss: 0.123870
step: 1700, acc: 0.953125, loss: 0.104512
step: 1800, acc: 1.000000, loss: 0.021029
step: 1900, acc: 0.976562, loss: 0.060067
step: 2000, acc: 1.000000, loss: 0.021355
step: 2100, acc: 0.960938, loss: 0.109562
step: 2200, acc: 1.000000, loss: 0.024127
step: 2300, acc: 0.984375, loss: 0.063855
step